# Dev notebook

I use this notebook to develop my implementation of a simple and minimal neural network framework.

Inspiration for this network is drawn from the book [Neural Networks from Scratch (NNFS)](https://nnfs.io/) & [Pytorch's implementation](https://pytorch.org/)

Notes:
- The work on experimenting with backpropagation has been moved to another notebook to keep this one minimal
- I have chosen to use a similar naming convention to that used by pytorch (why reinvent the wheel), this has the benefit of ensuring that when we compare implementations the architecture of the networks is the same. 

In [27]:
# Import for dev and testing
import nnfs 
import numpy as np

import matplotlib.pyplot as plt

from abc import ABC, abstractmethod
from nnfs.datasets import spiral_data
from typing import List

nnfs.init()

## Defining Base Module

This is a module that contains all the base attributes and function needed by every class in the framework

In [28]:
class Module(ABC):
    """Base class for all classes in frame work to ensure the same attributes and common function names."""

    def __init__(self) -> None:
        # Attributes to hold input and outputs
        self.input = None
        self.output = None
    
    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def backward(self):
        pass

##  Defining layers 

In this section We define and test the layers

Notes:
- since we intend to use ReLU as one of our activation functions we will use the He weight initialization method as described in https://arxiv.org/abs/1502.01852

### Linear Layer

In [29]:
class LinearLayer(Module):
    """Linear transformation layer of the type o = ixW + b,
    
    where I is the incoming vector, W is the layers weight matrix, b is bias vector and o is the dot product of the 
    i and W plus the bias
    
    Args:
        in_features (int): the size of the input features 
        out_features (int): the size of the output features
        
    Attributes:
        weights (np_array) numpy array of in_features x n_neurons
        biases  (np_array) numpy array of 1 x n_neurons
        inputs  (np_array) numpy array of latest batch of inputs
        inputs  (np_array) numpy array of latest batch of outputs
        d_w     (np_array) The current gradients with respect to the weights 
        d_x     (np_array) The current gradients with respect to the inputs
        d_b     (np_array) The current gradients with respect to the biases
    """

    def __init__(self, in_features, out_features, 
                 lambda_l1_weight=0, lambda_l1_bias=0, 
                 lambda_l2_weight=0, lambda_l2_bias=0) -> None:
        super().__init__()
        # initializing weights and biases 
        #self.weights = np.random.normal(0.0, np.sqrt(2/in_features), (in_features, out_features))
        # Using a simpler initialization  for testing 
        self.weights = 0.01 * np.random.randn(in_features, out_features)
        self.bias = np.zeros((1, out_features))
        # initializing regularization lambdas
        self.lambda_l1_weight = lambda_l1_weight
        self.lambda_l1_bias = lambda_l1_bias
        self.lambda_l2_weight = lambda_l2_weight
        self.lambda_l2_bias = lambda_l2_bias

    def forward(self, inputs):
        # Saving inputs for backward step
        self.input = inputs
        self.output = np.dot(inputs, self.weights) + self.bias
        return self.output

    def l1_backward_w(self):
        d_l1 = np.ones_like(self.weights) 
        d_l1[self.weights < 0] = -1
        return d_l1 * self.lambda_l1_weight

    def l1_backward_b(self):
        d_l1 = np.ones_like(self.bias) 
        d_l1[self.bias < 0] = -1
        return d_l1 * self.lambda_l1_bias

    def l2_backward_w(self):
        return 2 * self.lambda_l2_weight  * self.weights
        
    def l2_backward_b(self):
        return 2 * self.lambda_l2_bias  * self.bias

    def backward(self, d_vals):
        """Backpropagation  of the linear function

        Args:
            d_vals (np_array) array of derivatives from the previous layer/function.
        """
        self.d_w = np.dot(self.input.T, d_vals) + self.l1_backward_w() + self.l2_backward_w()
        self.d_x = np.dot(d_vals, self.weights.T)
        self.d_b = np.sum(d_vals, axis=0, keepdims=True) + self.l1_backward_b() + self.l2_backward_b()

    def regularization_loss(self):
        loss = 0
        # L1 weight 
        loss += self.lambda_l1_weight * np.sum(np.abs(self.weights))
        # L1 bias
        loss += self.lambda_l1_bias * np.sum(np.abs(self.bias))
        # L2 weight 
        loss += self.lambda_l2_weight * np.sum(self.weights * self.weights)
        # L1 bias
        loss += self.lambda_l2_bias * np.sum(self.bias * self.bias)

        return loss


#### Testing Linear Layer

In [30]:
# The sample data is a list of coordinate, ie two points 
# The layer therefore will take 2 inputs 
# We have given it 3 out features (3 neurons) so we expect to see an output with the shape (n_samples*n_neurons, n_neurons)
# In out case that should be (300, 3) 
X, _ = spiral_data(samples=100, classes=3)
linear1 = LinearLayer(2, 3)
output = linear1.forward(X)
print(output[:10, :])

assert output.shape == (300,3)


[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.0475188e-04  1.1395361e-04 -4.7983500e-05]
 [-2.7414842e-04  3.1729150e-04 -8.6921798e-05]
 [-4.2188365e-04  5.2666257e-04 -5.5912682e-05]
 [-5.7707680e-04  7.1401405e-04 -8.9430439e-05]
 [-3.5430698e-04  3.5025488e-04 -2.3363481e-04]
 [-8.9267001e-04  1.0767876e-03 -1.9453237e-04]
 [-9.3350781e-04  1.0723802e-03 -3.1227397e-04]
 [-1.1243758e-03  1.3112801e-03 -3.3629674e-04]
 [-1.3386955e-03  1.6200906e-03 -2.8101794e-04]]


## Defining Activation Functions

### ReLu

$$y = \begin{cases}
   x &x> 0 \\
   0 & otherwise
\end{cases} $$

In [31]:
class ReLU(Module):
    """Applies Rectified linear Unit function to vector."""
    def __init__(self) -> None:
        # initializing attributes needed for backwards 
        super().__init__()
        self.d_relu = None
    
    def forward(self, x):
        # storing inputs needed for backwards 
        self.inputs = x
        self.output = np.maximum(x, 0)
        return self.output
    
    def backward(self, d_vals):
        self.d_relu = d_vals.copy()
        self.d_relu[self.inputs <= 0] = 0

#### Testing ReLU

In [32]:
i = [-2, 3, 4, 0, 0.1, -44]
test_relu = ReLU()

# Checking values are as expected 
assert np.all(np.array([0., 3, 4, 0, 0.1, 0.]) == test_relu.forward(i))

test_relu.output

array([0. , 3. , 4. , 0. , 0.1, 0. ])

### Softmax

$$\text{softmax}(x)_i = \frac{exp(x_i)}{\sum_{j}^{ }exp(x_j))}$$

The soft max represents the confidence score for each output class and adds up to 1.

In [33]:
class Softmax(Module):
    """Applies Softmax function to input matrix."""

    def __init__(self) -> None:
        super().__init__()
        self.confidence_scores = None

    def forward(self, x):
        # exponenets of each value
        exp_vals = np.exp(x - np.max(x, axis=1, keepdims=True))
        exp_sum = np.sum(exp_vals, axis=1, keepdims=True)
        # Normalization to get the proabilities 
        self.output = exp_vals/exp_sum
        return self.output

    def _backward(self, d_vals):
        # Initialize array for gradients wrt to inputs
        self.d_soft = np.zeros_like(d_vals)
        
        _iter = enumerate(zip(self.output, d_vals))
        for i, conf_score, d_val in _iter:
            # Flatten confidence scores
            cs = conf_score.reshape(-1, 1)
            # Find the Jacobian matrix of the output 
            j_matrix = np.diagflat(cs) - np.dot(cs, cs.T)
            # get the gradient 
            self.d_soft[i] = np.dot(j_matrix, d_val)
    
    def backward(self, y_pred, y_true):
        """Does a the combined backward pass for CCE & Softmax as a single, faster step."""
        # Number of examples in the batch
        n = len(y_pred)

        # Getting descrete vals from one hot encoding 
        y_true = np.argmax(y_true, axis=1)
        
        self.d_soft = y_pred.copy()
        self.d_soft[range(n), y_true] -= 1
        self.d_soft = self.d_soft / n
        return self.d_soft


#### Testing Softmax

In [34]:
softmax = Softmax()
softmax.forward([[1,2,44]])

array([[2.11513104e-19, 5.74952226e-19, 1.00000000e+00]])

## Defining Loss - Categorical Cross-Entropy

$$ L_i = -\sum_j y_{i,j}\log(\hat{y}_{i,j}) $$

With taking one hot encoding into account we can simplify this down to:

$$ L_i = -y_{i,k}\log(\hat{y}_{i,k}) $$

where K is the index of the correct class

In [35]:
class CategoricalCrossEntropyLoss:
    """Calculates the CCE loss for a given set of predictions.
    This method expect a softmax output and one-hot encoded label mask
    
    y_pred (np_array): matrix of confidence scores of the prediction
    y_true (np_array): matrix of one-hot encoded true lables of the classes
    """
    def forward(y_pred, y_true):
        # Clipping and applying one hot encoded labels as mask 
        # to zero out scores corresponding to incorrect classes
        # We clip to make sure that none of the reaming classes are 0 or 
        # exactly 1 
        clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        corrected = np.sum(clipped*y_true, axis=1)
        # Taking the -ve log of the remaining confidence scores 
        negative_log = -np.log(corrected)
        return np.mean(negative_log)

    def backward(y_pred, y_true):
        """Backpropagation  of the CCE Loss

        Args:
            y_pred (np_array) array of predictions.
            y_true (np_array) array of correct labels.
        """
        return (-y_true/y_pred)/len(y_pred)

#### Testing CCE Loss

In [36]:
y_pred = np.array([[0.7, 0.1, 0.2], [0.1,0.5,0.4],[0.02,0.9,0.08]])
y_true = np.array([[1,0,0], [0,1,0], [0,1,0]])

loss_function = CategoricalCrossEntropyLoss
loss_function.forward(y_pred, y_true)

0.38506088005216804

## Defining Optimizers

### Stochastic Gradient Decent 

$$ \text{Update} = -\text{Learning Rate} \cdot \text{Gradient}$$

In [37]:
class SDG:
    """Stochastic Gradient Decent class used to update layer paramers
    The update is the -ve learning rate multiplied by the gradient calculated in the backward step.

    Attr:
        lr (float) Learning rate to scale the gradients by for the update
    """
    IMPLEMENTED = [LinearLayer]

    def __init__(self, learning_rate=1, decay=0., momentum=0.) -> None:
        self.lr = learning_rate
        self.clr = learning_rate # current learning rate
        self.decay = decay
        self.momentum = momentum
        self.iterations = 0

    def init_momentum(self, layers):
        for layer in layers:
            if not hasattr(layer, 'momentum_w'):
                layer.momentum_w = np.zeros_like(layer.weights)
                layer.momentum_b = np.zeros_like(layer.bias)

    def pre_update_step(self):
        decay_rate = 1/(1 + self.decay * self.iterations)
        self.clr = self.lr * decay_rate

    def get_updates(self, layer):
        return (
            -self.clr*layer.d_w,
            -self.clr*layer.d_b
        )

    def get_momentum_updates(self, layer):
        wu = (self.momentum * layer.momentum_w) - (self.clr * layer.d_w) 
        bu = (self.momentum * layer.momentum_b) - (self.clr * layer.d_b) 
        layer.momentum_w = wu
        layer.momentum_b = bu
        return (wu, bu)

    def update(self, layers):
        """Update a layers parameters.
        """
        # Test to make sure all layers supported
        if any(l for l in layers if type(l) not in self.IMPLEMENTED):
            unsupported = next(l for l in layers if type(l) not in self.IMPLEMENTED)
            raise NotImplementedError(f'SDG does not support {unsupported.__class__}')

        # pre update step
        if self.decay:
            self.pre_update_step()

        # On the first iteration using momentum initialize the layer momentums
        if self.iterations == 0 and self.momentum:
            self.init_momentum(layers)

        # Update step
        for layer in layers:

            if self.momentum:
                weight_u, bias_u = self.get_momentum_updates(layer)
            else:
                weight_u, bias_u = self.get_updates(layer)
            
            layer.weights += weight_u
            layer.bias += bias_u

        # post update
        self.iterations += 1 

### Adam

Short for Adaptive Momentum.

An extension to the Root mean square propagation (RSMprop) technique that adds in a bias correction mechanism used to correct the momentum and momentum caches.

To find the update with Adam we need to take the following steps:

1. Find momentum for the current step
2. Get corrected the momentum 
3. Update the cache with the square of the gradient 
4. Get the corrected cache 
5. Update weights 


In the first step we calculate the layer weight and bias momentums by:

$$ \text{Layer Momentum} = (\beta_1 \cdot \text{Layer Momentum}) + ((1 - \beta_1) \cdot gradient)$$ 

where $\beta_1$ is a hyper-parameter that allows us to apply fractions of the momentum and gradient at each step. 

To correct this we then divide the momentum by bias correction mechanism: 

$$ \text{Corrected Momentum} = \frac{\text{Layer Momentum}}{1 - \beta_1^{n+1}} $$

where $n$ is the number of the iteration/epoch and we add 1 to it to account for initializing it from 0

Next we update the cache for the weights and biases:

$$ \text{Cache} = (\beta_2 \cdot \text{Cache}) + ((1 - \beta_2) * \text{gradients}^2)$$

We once again correct, this time the cache, with Adam's bias correction mechanism:

$$ \text{Corrected Cache} = \frac{\text{Cache}}{{1 - \beta_2^{n+1}}}$$

Finally to update the weights we do the following:

$$ \text{Update} = \frac{\text{Current Learning Rate} \cdot \text{Corrected Momentum}}{\sqrt{\text{Corrected Cache}} + \epsilon} $$ 


In [38]:
class Adam:
    """Adam Optimizer"""

    IMPLEMENTED = [LinearLayer]

    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999) -> None:
        self.lr = learning_rate
        self.clr = learning_rate # current learning rate
        self.decay = decay
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.iterations = 0

    def pre_update_step(self):
        decay_rate = 1/(1 + self.decay * self.iterations)
        self.clr = self.lr * decay_rate

    def init_momentum(self, layers:List[LinearLayer]):
        for layer in layers:
            # Init momentum for weights
            layer.momentums_w = np.zeros_like(layer.weights)
            layer.cache_w = np.zeros_like(layer.weights)

            # Init momentums for biases
            layer.momentums_b = np.zeros_like(layer.bias)
            layer.cache_b = np.zeros_like(layer.bias)
            

    def update(self, layers:List[LinearLayer]):
        # pre update step
        if self.decay:
           self.pre_update_step()
        
        if self.iterations == 0:
            self.init_momentum(layers)

        # Update step
        for layer in layers:     
            ## Updating momentum 
            layer.momentums_w = self.beta_1 * layer.momentums_w + (1 - self.beta_1) * layer.d_w
            layer.momentums_b = self.beta_1 * layer.momentums_b + (1 - self.beta_1) * layer.d_b

            ## Correcting momentum 
            correction_bias_momentums = 1 - self.beta_1**(self.iterations +1)

            corrected_weights = layer.momentums_w / correction_bias_momentums
            corrected_bias    = layer.momentums_b / correction_bias_momentums

            ## Updating cache
            layer.cache_w = self.beta_2 * layer.cache_w + (1 - self.beta_2) * layer.d_w**2
            layer.cache_b = self.beta_2 * layer.cache_b + (1 - self.beta_2) * layer.d_b**2

            ## Correcting cache
            correction_bias_cache = 1 - self.beta_2**(self.iterations +1)

            corrected_cache_w = layer.cache_w / correction_bias_cache
            corrected_cache_b = layer.cache_b / correction_bias_cache

            ## Updating weights 
            layer.weights += -self.clr * corrected_weights / (np.sqrt(corrected_cache_w) + self.epsilon)

            ## Updating bias
            layer.bias    += -self.clr * corrected_bias / (np.sqrt(corrected_cache_b) + self.epsilon)
        
        # Post update step
        self.iterations += 1



## Defining Utility functions

### One-hot encoding function 

In [39]:
def one_hot_encode_index(y, n):
    return np.eye(n)[y]

#### Testing one hot masker

In [40]:
n=3
y_test = np.array([0,1,2, 1, 2])

one_hot_encode_index(y_test, n)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

### Define Accuracy 

In [41]:
def accuracy(y_pred, y_true):
    """Calculates the accuracy of a batch of predictions"""
    return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

## Defining Regularization 

Regularization methods are used to reduce generalization errors, With L1 and L2 regularization we calculate a penalty that we add to the loss if weight and biases are large. We want to see many neurons contribute to the evaluation rather than a few having a large impact.

L1 weight regularization: 

$$ L_{1w} = \lambda\sum_m|w_m| $$

L1 bias regularization: 

$$ L_{1b} = \lambda\sum_n|b_n| $$

L2 weight regularization: 

$$ L_{2w} = \lambda\sum w^2_m $$

L2 bias regularization: 

$$ L_{2b} = \lambda\sum_n b^2_n $$

Overall loss:

$$ \text{Loss} = \text{DataLoss} + L_{1w} + L_{1b} + L_{2w} + L_{2b} $$


To implement these changes we will need to modify the Linear Layer class.

(note: for backpropagation see backpropagation notebook)



## Integration Testing 

### Test with SDG

In [49]:
nnfs.init()
X, y = spiral_data(samples=100, classes=3)
y = one_hot_encode_index(y, 3)

# Initializing Network Components 
relu = ReLU()
softmax = Softmax()
cce_loss = CategoricalCrossEntropyLoss
optimizer = SDG(decay=1e-3, momentum=0.9)
linear1 = LinearLayer(2, 64, lambda_l2_weight=5e-4, lambda_l2_bias=5e-4)
linear2 = LinearLayer(64, 3)

update_layers = [linear1, linear2]

n_epochs = 10000

for epoch in range(n_epochs + 1):
    # Forward Pass
    linear1.forward(X) 
    relu.forward(linear1.output)
    linear2.forward(relu.output)
    y_pred = softmax.forward(linear2.output)

    # Calculating loss and regularized loss  
    loss = cce_loss.forward(y_pred, y)
    # Regularised loss
    rl = sum([l.regularization_loss() for l in update_layers if hasattr(l, 'regularization_loss')])
    regularized_loss = loss + rl

    #Calculating accuracy 
    acc = accuracy(y_pred, y) 

    # Printing results
    if not epoch % 100:
        print(f"Epoch:{epoch}, Loss:{loss:.3f}, Reg loss: {regularized_loss:.3f}, ({rl:.3f}), accuracy:{acc:.3f}")

    # Backward pass
    softmax.backward(y_pred, y)
    linear2.backward(softmax.d_soft)
    relu.backward(linear2.d_x)
    linear1.backward(relu.d_relu)

    # Optimization Step
    optimizer.update(update_layers)
    

Epoch:0, Loss:1.099, Reg loss: 1.099, (0.000), accuracy:0.360
Epoch:100, Loss:1.052, Reg loss: 1.055, (0.003), accuracy:0.447
Epoch:200, Loss:0.986, Reg loss: 1.009, (0.024), accuracy:0.500
Epoch:300, Loss:0.870, Reg loss: 0.910, (0.040), accuracy:0.570
Epoch:400, Loss:0.881, Reg loss: 0.964, (0.083), accuracy:0.597
Epoch:500, Loss:0.718, Reg loss: 0.798, (0.080), accuracy:0.657
Epoch:600, Loss:0.872, Reg loss: 0.957, (0.085), accuracy:0.567
Epoch:700, Loss:0.620, Reg loss: 0.711, (0.090), accuracy:0.700
Epoch:800, Loss:0.579, Reg loss: 0.673, (0.095), accuracy:0.743
Epoch:900, Loss:0.590, Reg loss: 0.684, (0.094), accuracy:0.740
Epoch:1000, Loss:0.646, Reg loss: 0.769, (0.123), accuracy:0.683
Epoch:1100, Loss:0.536, Reg loss: 0.639, (0.103), accuracy:0.773
Epoch:1200, Loss:0.556, Reg loss: 0.656, (0.100), accuracy:0.747
Epoch:1300, Loss:0.537, Reg loss: 0.638, (0.102), accuracy:0.747
Epoch:1400, Loss:0.519, Reg loss: 0.619, (0.100), accuracy:0.790
Epoch:1500, Loss:0.502, Reg loss: 0.6

In [43]:
# Validating the model 
X_test, y_test = spiral_data(samples=100, classes=3)
y_test = one_hot_encode_index(y_test, 3)

linear1.forward(X_test) 
relu.forward(linear1.output)
linear2.forward(relu.output)
y_pred = softmax.forward(linear2.output)

# Calculating loss and accuracy 
loss = cce_loss.forward(y_pred, y_test)
acc = accuracy(y_pred, y_test) 

print(f'Vallidation: Accuracy: {acc:.3f}, loss: {loss:.3f}')

Vallidation: Accuracy: 0.790, loss: 0.689


### Test with Adam

In [54]:
X, y = spiral_data(samples=100, classes=3)
y = one_hot_encode_index(y, 3)

# Initializing Network Components 
relu = ReLU()
softmax = Softmax()
cce_loss = CategoricalCrossEntropyLoss
optimizer = Adam(learning_rate=0.02, decay=5e-7)
linear1 = LinearLayer(2, 256, lambda_l2_weight=5e-4, lambda_l2_bias=5e-4)
linear2 = LinearLayer(256, 3)

update_layers = [linear1, linear2]

n_epochs = 10000

for epoch in range(n_epochs + 1):
    # Forward Pass
    linear1.forward(X) 
    relu.forward(linear1.output)
    linear2.forward(relu.output)
    y_pred = softmax.forward(linear2.output)

    # Calculating loss and regularized loss  
    loss = cce_loss.forward(y_pred, y)
    # Regularised loss
    rl = sum([l.regularization_loss() for l in update_layers if hasattr(l, 'regularization_loss')])
    regularized_loss = loss + rl

    #Calculating accuracy 
    acc = accuracy(y_pred, y) 

    # Printing results
    if not epoch % 100:
        print(f"Epoch:{epoch}, Loss:{loss:.3f}, Reg loss: {regularized_loss:.3f}, ({rl:.3f}), accuracy:{acc:.3f}")

    # Backward pass
    softmax.backward(y_pred, y)
    linear2.backward(softmax.d_soft)
    relu.backward(linear2.d_x)
    linear1.backward(relu.d_relu)

    # Optimization Step
    optimizer.update(update_layers)


Epoch:0, Loss:1.099, Reg loss: 1.099, (0.000), accuracy:0.353
Epoch:100, Loss:0.664, Reg loss: 0.721, (0.056), accuracy:0.740
Epoch:200, Loss:0.435, Reg loss: 0.531, (0.097), accuracy:0.860
Epoch:300, Loss:0.342, Reg loss: 0.449, (0.107), accuracy:0.873
Epoch:400, Loss:0.296, Reg loss: 0.403, (0.107), accuracy:0.913
Epoch:500, Loss:0.259, Reg loss: 0.364, (0.105), accuracy:0.930
Epoch:600, Loss:0.236, Reg loss: 0.337, (0.101), accuracy:0.930
Epoch:700, Loss:0.222, Reg loss: 0.320, (0.098), accuracy:0.937
Epoch:800, Loss:0.209, Reg loss: 0.312, (0.103), accuracy:0.943
Epoch:900, Loss:0.199, Reg loss: 0.298, (0.099), accuracy:0.947
Epoch:1000, Loss:0.193, Reg loss: 0.290, (0.096), accuracy:0.947
Epoch:1100, Loss:0.188, Reg loss: 0.282, (0.094), accuracy:0.947
Epoch:1200, Loss:0.182, Reg loss: 0.274, (0.091), accuracy:0.953
Epoch:1300, Loss:0.179, Reg loss: 0.268, (0.089), accuracy:0.953
Epoch:1400, Loss:0.174, Reg loss: 0.261, (0.087), accuracy:0.953
Epoch:1500, Loss:0.169, Reg loss: 0.2

In [53]:
# Validating the model 
X_test, y_test = spiral_data(samples=100, classes=3)
y_test = one_hot_encode_index(y_test, 3)

linear1.forward(X_test) 
relu.forward(linear1.output)
linear2.forward(relu.output)
y_pred = softmax.forward(linear2.output)

# Calculating loss and accuracy 
loss = cce_loss.forward(y_pred, y_test)
acc = accuracy(y_pred, y_test) 

print(f'Vallidation: Accuracy: {acc:.3f}, loss: {loss:.3f}')

Vallidation: Accuracy: 0.853, loss: 0.382
